In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import xml.etree.ElementTree as ET
import libfmp.b

Aqui vamos gerar as sequências de acordes explícitas do nosso corpus de audio.

In [7]:
songs = os.listdir('corpus/xml')
for song in songs:
    tree = ET.parse('corpus/xml/{}'.format(song))
    root = tree.getroot()
    song_title = song.split('.')[0]
    df_harmony = pd.DataFrame(columns=[song_title])

    part = root.findall('part')[0]
    measures = part.findall('measure')

    for measure in measures:
        chords = measure.findall('harmony')
        for chord in chords:
            try:
                note = chord.find('root/root-step').text

            except AttributeError: # se não achar a nota "root" do acorde, não tem acorde
                print('Measure: {}'.format(measure.attrib['number']), ' | ', 'No chords')
                continue

            if note:
                # alter é a alteração de um acorde (nem sempre existe): b ou #.
                alter = chord.find('root/root-alter')#.text

                # kind é o tipo do acorde (nem sempre existe): pode ser com 7a, 9a, 13a, por aí vai...
                kind = chord.find('kind')#.attrib['text']

                # bass é o baixo do acorde, quando diferente da tônica. Também nem sempre existe.
                bass = chord.find('bass/bass-step')#.text

                # bass_alter é e alteração do baixo, podendo ser b ou #. Também nem sempre existe.
                bass_alter = chord.find('bass/bass-alter')#.text

                attribs_dict = {'alter':alter, 
                                'kind':kind, 
                                'bass':bass, 
                                'bass_alter':bass_alter}

                for key, value in attribs_dict.items():
                    if value is None:
                        attribs_dict[key] = ''
                    elif key == 'kind':
                        try:
#                             print(song_title, measure.attrib['number'], note)
                            attribs_dict[key] = value.attrib['text']
                        except KeyError:
                            attribs_dict[key] = value.text#chord.find('kind').text
                    else:
                        attribs_dict[key] = value.text

                
                print(song_title, measure.attrib['number'], attribs_dict)
                
                measure_number = measure.attrib['number']
                chord = note + attribs_dict['alter'] + attribs_dict['kind']
                chord_bass = attribs_dict['bass']+attribs_dict['bass_alter']
                complete_chord = chord + '/' + chord_bass

                if complete_chord.endswith('/'):
                    complete_chord = complete_chord[:-1]

                df_chord = pd.DataFrame(columns=[song_title], index=[measure_number], data=complete_chord)
                df_harmony = df_harmony.append(df_chord)

    df_harmony.index.names = ['Measure']
    
    
    
    # tipo de compasso
    first_measure = measures[0]
    cima = first_measure.find('attributes').find('time/beats').text
    baixo = first_measure.find('attributes').find('time/beat-type').text

    compasso = cima + '/' + baixo
    
    # andamento
    andamento = 120
    
    # sync: acorde + minutagem
    duracao_seminima = 60/int(andamento)
    segs_compasso = (4/int(baixo)) * int(cima) * duracao_seminima
    df_harmony.index = df_harmony.index.astype(int)

    for compasso in df_harmony.index.unique():
        acordes = df_harmony.loc[[compasso]]
        n_acordes = len(acordes)
    #     print(compasso, n_acordes)
        inicio_compasso = (int(compasso) - 1) * segs_compasso

        if n_acordes == 1:
            tempos = inicio_compasso

        else:        
            tempos = np.linspace(inicio_compasso,
                                 inicio_compasso + segs_compasso,
                                 n_acordes+1)
            tempos = tempos[:-1]

        df_harmony.at[compasso, 'Time'] = tempos
        
        
    df_harmony.to_csv('corpus/chords_sequences/{}.csv'.format(song_title))
    print(song_title)

Sham Time 1 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 1 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 7 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 8 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 9 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 10 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 11 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 12 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 13 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 14 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 15 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 16 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 17 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 18 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Sham Time 1

TheOddCouple
JumpinWithSymphonySid 1 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid 3 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid 4 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid 5 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid 7 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid 9 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid 10 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid 11 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
JumpinWithSymphonySid
BeiMir 1 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
BeiMir 1 {'alter': '', 'kind': '', 'bass': '', 'bass_alter': ''}
BeiMir 2 {'alter': '-1', 'kind': '', 'bass': '', 'bass_alter': ''}
BeiMir 2 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
BeiMir 3 {'alter': '', 'kind': 'min7', 'bass': '

MessAround 21 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
MessAround 22 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
MessAround 23 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
MessAround 24 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
MessAround
Blues A La Mode 1 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 2 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 3 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 4 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 4 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 4 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 5 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 6 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Blues A La Mode 7 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
B

ThatIsWhyYouAreOverweight
HomeFires 33 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 37 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 38 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 39 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 40 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 41 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 43 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 44 {'alter': '', 'kind': 'dim', 'bass': '', 'bass_alter': ''}
HomeFires 45 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
HomeFires 45 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 46 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
HomeFires 46 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 47 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
HomeFires 49 {'alter': '-1', 'kind': '7', 

Afrodisia
stlouisblues 1 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 2 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 3 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 4 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 5 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 6 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 7 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 8 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 9 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 10 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 11 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 12 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
stlouisblues 12 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
stlouisblues 14 {'alter': '', 'kind': 'min7', '

...

SoftImpressions 4 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
SoftImpressions 5 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
SoftImpressions 5 {'alter': '', 'kind': '', 'bass': 'G', 'bass_alter': ''}
SoftImpressions 6 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
SoftImpressions 6 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
SoftImpressions 7 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
SoftImpressions 7 {'alter': '', 'kind': '', 'bass': 'G', 'bass_alter': ''}
SoftImpressions 8 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
SoftImpressions 8 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
SoftImpressions 9 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
SoftImpressions 9 {'alter': '', 'kind': '', 'bass': 'G', 'bass_alter': ''}
SoftImpressions 10 {'alter': '', 'kind': '', 'bass': 'F', 'bass_alter': '1'}
SoftImpressions 10 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
SoftImpressio

Cinnamon and Clove 36 {'alter': '', 'kind': 'dim', 'bass': '', 'bass_alter': ''}
Cinnamon and Clove 39 {'alter': '', 'kind': 'min', 'bass': '', 'bass_alter': ''}
Cinnamon and Clove 40 {'alter': '', 'kind': 'min', 'bass': 'G', 'bass_alter': '-1'}
Cinnamon and Clove 41 {'alter': '', 'kind': 'min', 'bass': 'F', 'bass_alter': ''}
Cinnamon and Clove 42 {'alter': '', 'kind': 'min', 'bass': 'E', 'bass_alter': ''}
Cinnamon and Clove 43 {'alter': '', 'kind': 'min', 'bass': '', 'bass_alter': ''}
Cinnamon and Clove 44 {'alter': '', 'kind': 'min', 'bass': 'G', 'bass_alter': '-1'}
Cinnamon and Clove 45 {'alter': '', 'kind': 'min', 'bass': 'F', 'bass_alter': ''}
Cinnamon and Clove 46 {'alter': '', 'kind': 'min', 'bass': 'E', 'bass_alter': ''}
Cinnamon and Clove 47 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
Cinnamon and Clove 48 {'alter': '', 'kind': '9', 'bass': '', 'bass_alter': ''}
Cinnamon and Clove 49 {'alter': '-1', 'kind': 'maj', 'bass': '', 'bass_alter': ''}
Cinnamon and Clov

Watermelon Man2 23 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Watermelon Man2 24 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Watermelon Man2 25 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Watermelon Man2 26 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Watermelon Man2 27 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Watermelon Man2 28 {'alter': '', 'kind': '13', 'bass': '', 'bass_alter': ''}
Watermelon Man2 29 {'alter': '-1', 'kind': '13', 'bass': '', 'bass_alter': ''}
Watermelon Man2 29 {'alter': '-1', 'kind': '13', 'bass': '', 'bass_alter': ''}
Watermelon Man2 29 {'alter': '', 'kind': '13', 'bass': '', 'bass_alter': ''}
Watermelon Man2 30 {'alter': '', 'kind': '13', 'bass': '', 'bass_alter': ''}
Watermelon Man2 30 {'alter': '', 'kind': '13', 'bass': '', 'bass_alter': ''}
Watermelon Man2 31 {'alter': '-1', 'kind': '13', 'bass': '', 'bass_alter': ''}
Watermelon Man2 31 {'alter': '-1', 'kind': '13', 'bass': '', 'bass_alte

psychedelic sally C
Weasil, The 2 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Weasil, The 4 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Weasil, The 6 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
Weasil, The 8 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Weasil, The 10 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Weasil, The 11 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Weasil, The 12 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
Weasil, The
InnerCityBlues 4 {'alter': '-1', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
InnerCityBlues 12 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
InnerCityBlues 16 {'alter': '-1', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
InnerCityBlues 28 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
InnerCityBlues 32 {'alter': '-1', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
InnerCityBlues
Jean De Fleur 1 {'alter': '1', 'kind': 'min7', 'bass': '',

I Wish
MoonRiver 1 {'alter': '', 'kind': 'maj7', 'bass': '', 'bass_alter': ''}
MoonRiver 2 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
MoonRiver 3 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
MoonRiver 4 {'alter': '', 'kind': 'maj7', 'bass': '', 'bass_alter': ''}
MoonRiver 5 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
MoonRiver 6 {'alter': '', 'kind': 'maj7', 'bass': '', 'bass_alter': ''}
MoonRiver 7 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
MoonRiver 8 {'alter': '', 'kind': '7', 'bass': '', 'bass_alter': ''}
MoonRiver 9 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
MoonRiver 10 {'alter': '', 'kind': '7', 'bass': 'G', 'bass_alter': ''}
MoonRiver 11 {'alter': '', 'kind': 'maj7', 'bass': '', 'bass_alter': ''}
MoonRiver 12 {'alter': '-1', 'kind': '7', 'bass': '', 'bass_alter': ''}
MoonRiver 13 {'alter': '', 'kind': 'min7', 'bass': '', 'bass_alter': ''}
MoonRiver 13 {'alter': '', 'kind': 'min7', 'bass': 'G', 'bass_alt